In [1]:
import pandas as pd
import numpy as np

In [5]:
#Construcao da Dimensao Data
#Dimensao Data nao tem tabela de origem - feita de raiz no python com intervalo de tempo correspondente ao identificado na fase1
#Novo Dataframe
dfdata=pd.DataFrame(columns=['IDData','Ano'])

#Criacao da Coluna Ano com 66 periodos e frequencia anual
dfdata['Ano']=pd.period_range("1/1/1956", freq="Y", periods=66)

#Criacao da coluna ID atraves do uso do indice do pandas
dfdata['IDData']=dfdata.index+1

#Passar Ano do tipo range para tipo int de forma a poder fazer operacoes sobre a coluna
dfdata['Ano'] = dfdata['Ano'].astype(str).astype(int)

#Criacao da coluna Metade da Decada atraves de condicoes com recurso ao resto inteiro da divisao por 10
conditions2=[((dfdata['Ano']%10)<5),(dfdata['Ano']%10)>=5]
values2=['Primeira Metade da Decada','Segunda Metade da Decada']
dfdata['MetadeDaDecada']=np.select(conditions2,values2)

#Criacao da coluna Decada com condicoes de intervalo
conditions=[(dfdata['Ano']<1960),
           (dfdata['Ano']>=1960) & (dfdata['Ano']<1970),
           (dfdata['Ano']>=1970) & (dfdata['Ano']<1980),
           (dfdata['Ano']>=1980) & (dfdata['Ano']<1990),
           (dfdata['Ano']>=1990) & (dfdata['Ano']<2000),
           (dfdata['Ano']>=2000) & (dfdata['Ano']<2010),
           (dfdata['Ano']>=2010) & (dfdata['Ano']<2020),
           (dfdata['Ano']>=2020)]
values=[1950,1960,1970,1980,1990,2000,2010,2020]
dfdata['Decada']=np.select(conditions,values)

#Criacao da coluna Seculo com condicoes de intervalo
conditions1=[(dfdata['Ano']<2000),dfdata['Ano']>=2000]
values1=[20,21]
dfdata['Seculo']=np.select(conditions1,values1)

#Resultado Final
dfdata.to_csv("DimData_final.csv", index=False)

In [6]:
#Construcao da Junk Dimension - sem dataset de origem

#Novo dataframe
dfjunk=pd.DataFrame(columns=['IDJunk','VerificacaoVizinho','VerificacaoVoto'])

#Criacao de valores de colunas
dfjunk['VerificacaoVizinho']=['PaisesVizinhos','PaisesVizinhos','PaisesVizinhos','PaisesNaoVizinhos','PaisesNaoVizinhos','PaisesNaoVizinhos']
dfjunk['VerificacaoVoto']=['Televoto','VotoJuri','VotoNaoDiscriminado','Televoto','VotoJuri','VotoNaoDiscriminado']

#Criacao da coluna ID atraves do uso do indice do pandas
dfjunk['IDJunk']=dfjunk.index+1

#Resultado Final
dfjunk.to_csv("DimJunk_final.csv", index=False)

In [7]:
#Construcao da Dimensao Conflitos Simplificada
#Esta dimensao funciona como uma junk dimension

#Novo dataframe
dfconfli=pd.DataFrame(columns=['IDConflito','VerificacaoConflito','LocalizacaoConflito'])

#Criacao de valores de colunas
dfconfli['VerificacaoConflito']=['NaoEnvolvidoEmConflito','EnvolvidoEmConflito','EnvolvidoEmConflito','EnvolvidoEmConflito','EnvolvidoEmConflito']
dfconfli['LocalizacaoConflito']=['NA','ConflitoNumParticipanteEurovisao','ConflitoNumParticipanteEurovisao','ConflitoNumNaoParticipanteEurovisao','ConflitoNumNaoParticipanteEurovisao']
dfconfli['AtivoEurovisao']=['NA','AtivoDuranteEurovisao','NaoAtivoDuranteEurovisao','AtivoDuranteEurovisao','NaoAtivoDuranteEurovisao']

#Criacao da coluna ID atraves do uso do indice do pandas
dfconfli['IDConflito']=dfconfli.index+1

#Resultado Final
dfconfli.to_csv("DimJunkConflito_final.csv", index=False)

In [8]:
#Construcao da Dimensao Localizacao
#2 Dataset origem: localizacao e Vizinhos
dflocal_1=pd.read_excel('Localizacao.xlsx')
dfvizinhos=pd.read_excel('Vizinhos.xlsx', sheet_name='LB_Eurovision')

#Novo dataframe
dflocal=pd.DataFrame(columns=['IDLocalizacao','Pais'])

#Alteracao de nome da coluna para facilitar o merge
dfvizinhos.rename(columns = {'Name':'English Name'}, inplace = True)

#Eliminacao de colunas que nao vao ser utilizadas
dfvizinhos.drop(columns=['Neighbour10', 'Neighbour9','Neighbour8', 'Neighbour7','Neighbour6', 'Neighbour5','Neighbour4', 'Neighbour3','Neighbour2','Neighbour1','Unnamed: 3', 'Unnamed: 14'])

#Merge dos dois dataset de origem
result = pd.merge(dflocal_1, dfvizinhos, on=["English Name"])

#Composicao da dimensao Localizacao
dflocal['Pais']=result['English Name']
dflocal['Regiao']=result['Region of the territory']
dflocal['Continente']=result['Continent of the territory']
dflocal['IDLocalizacao']=dflocal.index+1
dflocal['NumeroDeVizinhos']=result['NrNeighboursinEurope']

#Resultado Final
dflocal.to_csv("DimLocalizacao_final.csv", index=False)

In [10]:
#Construcao da Dimensao Conflitos
#Esta dimensao nao foi utilizada diretamente no data warehouse mas utilizada para verificar os ID da JunkConflict utilizados

#Esta dimensao utiliza dois dataset de origem: Localizacao e Conflitos. Como a dimensao localizacao tinha sido ja criada
#optamos por utilizar esta em vez do dataset original pois continha menos dados que nao eram uteis facilitando a limpeza
dfl=pd.read_csv('DimLocalizacao.csv')
dfc=pd.read_excel('Conflitos.xlsx')
dfconflito=pd.DataFrame(columns=['IDConflito'])

#Criacao das colunas de importacao direta dos datasets
dfconflito['LocalizacaoConflito']=dfc['Conflict Location']
dfconflito['IDConflito']=dfconflito.index+1
dfconflito['ParticipaEurovisao']=np.where(dfconflito['LocalizacaoConflito'].isin(dfl['Pais']),'ParticipanteEurovisao','NaoParticipaNaEurovisao')
dfconflito['NomeConflito']=dfc['Conflict']
dfconflito['DataInicio']=dfc['Start date']
dfconflito['DataFim']=dfc['End date']

#Passagem das datas para Ano de forma a respeitar a granularidade da TF
dfconflito['AnoInicio']=pd.DatetimeIndex(dfconflito['DataInicio']).year
dfconflito['AnoFim']=pd.DatetimeIndex(dfconflito['DataFim']).year

#Continuacao da criacao das colunas
dfconflito['Participante']=dfc['Participant 1']

#Quando o conflito ainda nao esta concluido, toma o valor de 9999 para ser possivel passar o tipo da coluna para inteiro
#e realizar operacoes numericas
dfconflito['AnoFim'].fillna('9999',inplace=True)
dfconflito.AnoFim = dfconflito.AnoFim.astype(int)

#Criacao de Condicoes para o conflito ativo ou nao durante a eurovisao
conditions=[(dfconflito['DataFim']-dfconflito['DataInicio']).dt.days>365,
            (pd.DatetimeIndex(dfconflito['DataInicio']).year==(pd.DatetimeIndex(dfconflito['DataFim']).year)) & (pd.DatetimeIndex(dfconflito['DataInicio']).month<=5) & (pd.DatetimeIndex(dfconflito['DataFim']).month>=5),
            (pd.DatetimeIndex(dfconflito['DataInicio']).year==(pd.DatetimeIndex(dfconflito['DataFim']).year)) & (pd.DatetimeIndex(dfconflito['DataInicio']).month<=1) & (pd.DatetimeIndex(dfconflito['DataFim']).month<=4),
            (pd.DatetimeIndex(dfconflito['DataInicio']).year==(pd.DatetimeIndex(dfconflito['DataFim']).year)) & (pd.DatetimeIndex(dfconflito['DataInicio']).month>=6) & (pd.DatetimeIndex(dfconflito['DataFim']).month<=12),
            (pd.DatetimeIndex(dfconflito['DataInicio']).year!=(pd.DatetimeIndex(dfconflito['DataFim']).year)) & (pd.DatetimeIndex(dfconflito['DataInicio']).month>5) & (pd.DatetimeIndex(dfconflito['DataFim']).month<5),
            (pd.DatetimeIndex(dfconflito['DataInicio']).year!=(pd.DatetimeIndex(dfconflito['DataFim']).year)) & (pd.DatetimeIndex(dfconflito['DataInicio']).month<=5) & (pd.DatetimeIndex(dfconflito['DataFim']).month>=5),
            (pd.DatetimeIndex(dfconflito['DataInicio']).year!=(pd.DatetimeIndex(dfconflito['DataFim']).year)) & (pd.DatetimeIndex(dfconflito['DataInicio']).month>=5) & (pd.DatetimeIndex(dfconflito['DataFim']).month>=5),
            (dfconflito['AnoFim'])==999]
values=['AtivoDuranteEurovisao','AtivoDuranteEurovisao','NaoAtivoDuranteEurovisao','NaoAtivoDuranteEurovisao','AtivoDuranteEurovisao','NaoAtivoDuranteEurovisao','AtivoDuranteEurovisao','AtivoDuranteEurovisao']
dfconflito['EstadoDuranteEurovisao']=np.select(conditions,values) 

#Alterar os valores de 9999 para vazio de forma a facilitar a importacao no Postgresql
dfconflito['AnoFim'] = dfconflito.AnoFim.replace(9999,'')

#Apagar as colunas de data que ja nao sao necessarias
dfconflito=dfconflito.drop(columns=['DataInicio', 'DataFim'])

#Resultado Final
dfconflito.to_csv('DimConflito_final.csv')

In [16]:
#Construcao da Dimensao Musica
#Esta dimensao teve como tabela de origem o dataset musica

df = pd.read_excel('Musica.xlsx')
dfmusica=pd.DataFrame(columns=['IDMusica','Musica'])

#Preenchimento dos campos de importacao direta
dfmusica['Musica']=df['Value.Song']
dfmusica['IDMusica']=dfmusica.index+1
dfmusica['Lingua']=df['EnglishNonEnglish']
dfmusica['Classificacao']=df['Value.Pl.']

#Alterar os valores de NaN para vazio de forma a facilitar a importacao no Postgresql
dfmusica.fillna('',inplace=True)

#Preenchimento dos campos de importacao direta
dfmusica['OrdemAtuacao']=df['RunningOrder']

#Alterar os valores de NaN para vazio de forma a facilitar a importacao no Postgresql
dfmusica.fillna('',inplace=True)

#Preenchimento dos campos de importacao direta
dfmusica['OrdemAtuacaoNormalizada']=df['Ropercent']
dfmusica['Pontuacao']=df['Value.Sc.']

#Alterar os valores de NaN para vazio de forma a facilitar a importacao no Postgresql
dfmusica.fillna('',inplace=True)

#Resultado final
dfmusica.to_csv("DimMusica_final.csv", index=False)